The datastream from the QuarkNet DAQ board output into an ASCII format where each line contains 16 words. 

For example:
```
687C4047 80 00 2B 00 00 00 00 00 67037CB8 000322.027 180516 A 03 0 +0053
687C4047 00 00 00 00 3A 00 00 00 67037CB8 000322.027 180516 A 03 0 +0053
687C4048 00 00 00 28 00 00 00 00 67037CB8 000322.027 180516 A 03 0 +0053
687C4048 00 00 00 00 00 36 00 00 67037CB8 000322.027 180516 A 03 0 +0053
```

First let's define some classes for TMC counts, DAQ status, and DAQ lines:

In [125]:
import string

class TMCCount:
    def __init__(self, word):
        self.word = word
        self.bin_word = format(int(word,16), '08b')
        
        # bits 0-4: TMC count of RE or FE
        # bit 5: channel edge tag (1: valid edge, 0: no edge)
        # bit 6: not used, always 0
        # bit 7: [rising edge] trigger tag (1: new trigger; start of new event, 0: follow-up data of new event)
        # bit 7: [falling edge] not used, always 0
        
        self.edge_count = self.bin_word[3:]
        
        if ( self.bin_word[2] == '1' ):
            self.edge_valid = True
        else:
            self.edge_valid = False

        self.bit_7 = self.bin_word[0]
        
    def dump(self):
        output = ' TMCCount:'
        output += '  edge_count=' + str(self.edge_count)
        output += '  edge_valid=' + str(self.edge_valid)
        output += '  bit_7=' + str(self.bit_7)
        print(output)

        
class DAQStatus:
    def __init__(self, word):
        self.word = word
        self.bin_word = format(int(word,16), '04b')
        
        # bit 0: 0: OK, 1: 1PPS interrupt pending
        # bit 1: 0: OK, 1: trigger interrupt pending
        # bit 2: 0: OK, 1: GPS data possibly corrupted while DAQ was busy
        # bit 3: 0: OK, 1: current or last 1PPS rate is not within 25 CPLD clock ticks
        self.bit_0 = self.bin_word[3]
        self.bit_1 = self.bin_word[2]
        self.bit_2 = self.bin_word[1]
        self.bit_3 = self.bin_word[0]
        
        if ( (int(self.bit_0) + int(self.bit_1) + int(self.bit_2) + int(self.bit_3)) > 0 ):
            self.is_ok = False
        else:
            self.is_ok = True
    
    
class DAQLine:
    def __init__(self, words):
        self.words = words
    
        # count of rising edge at input 0
        self.re_0 = TMCCount(words[1])
        # count of falling edge at input 0
        self.fe_0 = TMCCount(words[2])
                        
        # count of rising edge at input 1
        self.re_1 = TMCCount(words[3])
        # count of falling edge at input 1
        self.fe_1 = TMCCount(words[4])
        
        # count of rising edge at input 2
        self.re_2 = TMCCount(words[5])
        # count of falling edge at input 2
        self.fe_2 = TMCCount(words[6])
        
        # count of rising edge at input 3
        self.re_3 = TMCCount(words[7])
        # count of falling edge at input 3
        self.fe_3 = TMCCount(words[8])
    
        # 32-bit trigger count of the 25 MHz 
        # CPLD clock
        self.clock_count = words[0]
    
        # 32-bit CPLD count of the most recent 1 PPS
        # time mark from the GPS
        self.pps = words[9]
    
        # UTC time of most recent GPS receiver
        # data update
        # HHMMSS.mmm
        self.utc_time = words[10]
        
        # UTC data of most recent GPS receiver
        # data update
        # DDMMYY
        self.utc_date = words[11]
        
        # GPS valid/invalid flag
        if words[12] == 'A':
            self.gps_valid = True
        elif words[12] == 'V':
            self.gps_valid = False
        
        # Number of GPS satellites
        self.n_gps = int(words[13])
        
        # DAQ status flag
        self.daq_status = DAQStatus(words[14])
        
        # Time delay in milliseconds between
        # the 1 PPS pulse and the GPS data interrupt
        self.time_delay = words[15]

    def dump(self):
        output = 'DAQLine:'
        output += ' clock_count=' + str(self.clock_count)
        output += ' pps=' + str(self.pps)
        output += ' utc_time=' + str(self.utc_time)
        output += ' utc_date=' + str(self.utc_date)
        output += ' gps_valid=' + str(self.gps_valid)
        output += ' time_delay=' + str(self.time_delay)
        print(output)
        
        print(' rising edge 0:')
        self.re_0.dump()
        print(' faling edge 0:')
        self.fe_0.dump()
        

Then open a file and output some information from the first 4 lines:

In [126]:
ifile = open('data/qn/6148.2016.0518.0', 'r')       

for line in ifile.readlines()[0:4]:
    
    daq_line = DAQLine(string.split(line))
    
    if daq_line.daq_status.is_ok:
        daq_line.dump()

ifile.close()

DAQLine: clock_count=687C4047 pps=67037CB8 utc_time=000322.027 utc_date=180516 gps_valid=True time_delay=+0053
 rising edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=1
 faling edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
DAQLine: clock_count=687C4047 pps=67037CB8 utc_time=000322.027 utc_date=180516 gps_valid=True time_delay=+0053
 rising edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
 faling edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
DAQLine: clock_count=687C4048 pps=67037CB8 utc_time=000322.027 utc_date=180516 gps_valid=True time_delay=+0053
 rising edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
 faling edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
DAQLine: clock_count=687C4048 pps=67037CB8 utc_time=000322.027 utc_date=180516 gps_valid=True time_delay=+0053
 rising edge 0:
 TMCCount:  edge_count=00000  edge_valid=False  bit_7=0
 faling edge 0:
 TMCCount:  edge_count=00000  edge_v

Information taken from "QuarkNet Cosmic Ray Muon Detector User's Manual Series 6000 DAQ", January 2010 Version 1.1